This code is gathered from PySAL package developed by Saxon and colleaguages (2019).

Please refer here (https://pysal.org/access/index.html) to examine the documentation. 

In [ ]:
pip install geopandas

In [ ]:
pip install matplotlib

In [ ]:
pip install logging

In [ ]:
pip install access

In [ ]:
from access import Access, weights, Datasets
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# Calling the mean coordinates shapefile and population counts per neighborhood as the demand dataset
ist_dem =  gpd.read_file('/Users/aybikesahinoglu/Desktop/pl/Mean.shp')

In [3]:
ist_dem.head()

,ID,Population,geometry
0,564,13957,POINT (682184.024 4534009.117)
1,565,40485,POINT (681389.483 4532090.955)
2,566,9493,POINT (682391.786 4533384.149)
3,567,14750,POINT (681819.597 4533283.992)
4,560,32285,POINT (679353.105 4533642.948)


In [4]:
ist_dem.dtypes

ID               int64
Population       int64
geometry      geometry
dtype: object

In [5]:
# Calling the hospital locations and the number of available doctors as the supply dataset
ist_doc = gpd.read_file('/Users/aybikesahinoglu/Desktop/pl/Hospitals.shp')

In [6]:
ist_doc.head()

,ID,Doctors,geometry
0,1,2,POINT (670297.654 4539018.535)
1,2,185,POINT (667367.253 4546370.020)
2,3,53,POINT (659960.838 4545104.571)
3,4,10,POINT (660482.490 4544627.809)
4,5,12,POINT (660747.181 4544734.127)


In [7]:
# All items (except for the geometry) need to be integers
ist_doc.dtypes

ID             int64
Doctors        int64
geometry    geometry
dtype: object

In [8]:
# Calling the travel cost matrix created by OD Matrix via the QNEAT3 plugin on QGIS
travel_cost = pd.read_csv('/Users/aybikesahinoglu/Desktop/pl/imp/TTCM.csv')

df_travel_c = pd.DataFrame(travel_cost)

In [9]:
df_travel_c.head()

,origin_id,destination_id,entry_cost,network_cost,exit_cost,total_cost
0,564,1,93.004295,11634.311048,45.930121,11773.245464
1,564,2,93.004295,11914.478923,2.475013,12009.958232
2,564,3,93.004295,11780.939773,2.774950,11876.719018
3,564,4,93.004295,11996.528799,15.258979,12104.792074
4,564,5,93.004295,11737.404796,0.461915,11830.871006


In [10]:
df_travel_c.dtypes

origin_id           int64
destination_id      int64
entry_cost        float64
network_cost      float64
exit_cost         float64
total_cost        float64
dtype: object

In [11]:
# Adjusting the parameters of the access class
A = Access(demand_df            = ist_dem,
           demand_index         = 'ID',
           demand_value         = 'Population',
           supply_df            = ist_doc,
           supply_index         = 'ID',
           supply_value         = 'Doctors',
           cost_df              = df_travel_c,
           cost_origin          = 'origin_id',
           cost_dest            = 'destination_id',
           cost_name            = 'total_cost')

In [12]:
# Defining a gaussian weight to give more penalites that take longer to reach than the defined treshold, which is 90 minutes in this instance
gaussian = weights.gaussian(90)

In [13]:
# Initializing RAAM and defining the time treshold
raam_analysis = A.raam(name = "raam", tau = 90)

In [14]:
df_ra = pd.DataFrame(raam_analysis)

df_ra

,raam_Doctors
ID,
1,365.356340
2,363.013599
3,42.690444
4,8.216704
5,38.221639
...,...
952,98.538377
953,122.365294
954,104.563431


In [15]:
# Scoring the raam results 
raam_scores = A.score(name="raam",col_dict={"raam_Doctors": 1})

In [16]:
df_raams = pd.DataFrame(raam_scores)
df_raams

,raam
ID,
1,12.134626
2,12.056816
3,1.417883
4,0.272902
5,1.269460
...,...
952,3.272767
953,4.064134
954,3.472878


In [17]:
# Saving the results on desktop 
df_raams.to_csv('/Users/aybikesahinoglu/Desktop/RAAMSCORES90.csv',index=True)